In [ ]:
from hydromt_sfincs import SfincsModel
from hydromt.config import configread
from hydromt.log import setuplog
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import pandas as pd
import datetime
from hydromt_sfincs import utils
import xarray as xr


In [ ]:
main_dir = r"c:\Users\winter_ga\Offline_data\projects\InterTwin\FDZ_model"
run_name = "model_20m_final"      
manning_sea=0.02
wl_csv = r"c:\Users\winter_ga\Offline_data\projects\InterTwin\Wasserstaende\pegelonline-barhft-W-20231015-20231023.csv"
# offset of tide gauge from local datum
waterlevel_offset = -498
inifile = r"c:\Users\winter_ga\Offline_data\projects\InterTwin\Geodaten\zsini_regridded_20m_final.tif"

logger = setuplog()
opt = configread(os.path.join(main_dir,"fdz_base.yml"))

sf = SfincsModel(data_libs=r"c:\Users\winter_ga\Offline_data\projects\InterTwin\FDZ_model\data.yml", root=os.path.join(main_dir,run_name), mode='w+', logger=logger)
sf.config

In [ ]:
# Make model
sf.setup_config(
    tref= "20231015 000000",
    tstart= "20231015 070000",
    tstop= "20231023 000000",
    dtout= 10400.0,
    dthisout= 600.0,
    dtmaxout= 259200.0,
    zsini=-1.0,
    tspinup=86400,
    
)
sf.setup_grid_from_region(
    region = {'geom': r'c:\Users\winter_ga\Offline_data\projects\InterTwin\FDZ_model\mask_25833.geojson'},
    res = 20,
    rotated = False,
    crs = 25833,
)
# sf.setup_dep(**opt['setup_dep'])
# sf.setup_subgrid(**opt['setup_dep'])

# Add depth information to modelgrid based on these chosen datasets
datasets_dep = [{"elevtn": "MVtopo_masked", "zmin": 0.01},{"elevtn": "ELC_INSPIRE"},{"elevtn": "merit_hydro"}]
dep = sf.setup_dep(datasets_dep=datasets_dep)

sf.setup_mask_active(
    include_mask= r"c:\Users\winter_ga\Offline_data\projects\InterTwin\FDZ_model\mask_25833.geojson",
    zmax= -50.0
)
sf.setup_mask_bounds(
    btype="waterlevel",
    zmax= 0
)


# dep = sf.setup_subgrid(
#     datasets_dep=datasets_dep,
#     # manning_land=0.04,
#     # manning_sea=manning_sea, 
#     # rgh_lev_land=0.0,
#     write_dep_tif=True,
#     write_man_tif=True,
#     nr_subgrid_pixels=4,
#     )

# set up roughness 
sf.setup_manning_roughness(
    manning_land=0.04,
    manning_sea=manning_sea,
    rgh_lev_land=0,  # the minimum elevation of the land
)

sf.config

In [ ]:
# set initial water level to zero, except over land set it to -1
ini_ds = xr.open_dataset(inifile, engine="rasterio")
ini_ds["zsini"] = ini_ds["band_data"]
ini_ds = ini_ds.drop(["band_data"])

sf.set_states(data=ini_ds["zsini"])

sf.set_config("inifile","sfincs.ini")

In [ ]:
# set up water level boundary point (Barhoeft)
x = [372368.1]
y = [6033653.6]

# add to Geopandas dataframe as needed by HydroMT
pnts = gpd.points_from_xy(x, y)
index = [1]  # NOTE that the index should start at one
bnd = gpd.GeoDataFrame(index=index, geometry=pnts, crs=sf.crs)

display(bnd)

# add waterlevel forcing
# we use measured timeseries
df = pd.read_csv(wl_csv, header=0,delimiter=";")
df=df.rename(columns={"timestamp":"time", "value":1})
df[1]=pd.to_numeric(df[1])

time = pd.to_datetime(df["time"])

# and the actual water levels, in this case for input location 1 a water level rising from 0 to 2 meters and back to 0:
bzs = ((df[1]+waterlevel_offset)/100).to_list()

bzspd = pd.DataFrame(index=time, columns=index, data=bzs)
display(bzspd)

# Actually add it to the SFINCS model class:
sf.setup_waterlevel_forcing(timeseries=bzspd, locations=bnd, buffer=1000)

# NOTE: the waterlevel forcing data is now stored in the sf.forcing dictionary
sf.forcing.keys()
sf.plot_forcing(fn_out="forcing.png")

In [ ]:
# setup observation points 
from shapely.geometry import Point
d = {'Station': ["Barhoeft","Barth","Althagen","Barth2","Wiek-East","Wiek-South"], 
     'geometry': [
         Point(371100.0, 6034300.0), 
         Point(352691.1, 6027443.5),
         Point(332296.2, 6027473.9), 
         Point(345640.0, 6031872.0),
         Point(343260.0, 6030788.0),
         Point(352715.0, 6028084.0),
         ]}
gdf = gpd.GeoDataFrame(d, crs="EPSG:25833")
display(gdf)

sf.setup_observation_points(
    locations=gdf, merge=False
)
    

In [ ]:
# Write results
sf.write()
sf.plot_basemap()
plt.savefig(os.path.join(main_dir,run_name,'basemap.png'))

In [ ]:
# Add dikes

dike_file = r"c:\Users\winter_ga\Offline_data\projects\InterTwin\Geodaten\all_dikes.gpkg"

# HydroMT function: get geodataframe from filename
gdf_floodwall = sf.data_catalog.get_geodataframe(
    dike_file, geom=sf.region, crs=sf.crs
)

# Add floodwall attributes to geodataframe
if (gdf_floodwall.geometry.type == "MultiLineString").any():
    gdf_floodwall = gdf_floodwall.explode()

# par1 is the overflow coefficient for weirs
gdf_floodwall["par1"] = 0.6

# HydroMT function: create floodwall
sf.setup_structures(structures=gdf_floodwall, stype="weir", merge=False)

run_name_dikes = run_name+"_dikes"      

sf.set_root(root=os.path.join(main_dir,run_name_dikes), mode="w+")
sf.write()
sf.plot_basemap()
plt.savefig(os.path.join(main_dir,run_name_dikes,'basemap.png'))


In [ ]:
# Add dikes with breaches

dike_file = r"c:\Users\winter_ga\Offline_data\projects\InterTwin\Geodaten\all_dikes_withbreaches.gpkg"

# HydroMT function: get geodataframe from filename
gdf_floodwall = sf.data_catalog.get_geodataframe(
    dike_file, geom=sf.region, crs=sf.crs
)

# Add floodwall attributes to geodataframe
if (gdf_floodwall.geometry.type == "MultiLineString").any():
    gdf_floodwall = gdf_floodwall.explode()

# par1 is the overflow coefficient for weirs
gdf_floodwall["par1"] = 0.6

# HydroMT function: create floodwall
sf.setup_structures(structures=gdf_floodwall, stype="weir", merge=False)

run_name_dikes = run_name+"_dikes_withbreaches"      

sf.set_root(root=os.path.join(main_dir,run_name_dikes), mode="w+")
sf.write()
sf.plot_basemap()
plt.savefig(os.path.join(main_dir,run_name_dikes,'basemap.png'))


In [ ]:
# Add dikes with breaches

dike_file = r"c:\Users\winter_ga\Offline_data\projects\InterTwin\Geodaten\all_dikes_withovertopping.gpkg"

# HydroMT function: get geodataframe from filename
gdf_floodwall = sf.data_catalog.get_geodataframe(
    dike_file, geom=sf.region, crs=sf.crs
)

# Add floodwall attributes to geodataframe
if (gdf_floodwall.geometry.type == "MultiLineString").any():
    gdf_floodwall = gdf_floodwall.explode()

# par1 is the overflow coefficient for weirs
gdf_floodwall["par1"] = 0.6

# HydroMT function: create floodwall
sf.setup_structures(structures=gdf_floodwall, stype="weir", merge=False)

run_name_dikes = run_name+"_dikes_overtopping"      

sf.set_root(root=os.path.join(main_dir,run_name_dikes), mode="w+")
sf.write()
sf.plot_basemap()
plt.savefig(os.path.join(main_dir,run_name_dikes,'basemap.png'))
